<a href="https://colab.research.google.com/github/rrwiren/ilmanlaatu-ennuste-helsinki/blob/main/Uusi_esik%C3%A4sittely_skripti_2025-04-11_1046.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
==========================================================================================
 TÄYDELLINEN SKRIPTI HELSINGIN SÄÄ- JA ILMANLAATUDATAN YHDISTÄMISEEN (FMI)
==========================================================================================

 Versio: 2.0
 Päivitetty: 2025-04-11 10:46 (BST)

 Skriptin tarkoitus:
 --------------------
 1. Ladata kolme erillistä CSV-datatiedostoa (2 x säädata Kaisaniemestä,
    1 x ilmanlaatudata Kalliosta) määritellyistä URL-osoitteista GitHubista.
    Nämä tiedostot oletetaan alun perin ladatun FMI:n avoimen datan palvelusta.
 2. Esikäsitellä jokainen tiedosto huolellisesti:
    - Muodostaa yhtenäisen, aikavyöhykkeet huomioimattoman ('naive') datetime-indeksin
      ('Timestamp') yhdistämällä Vuosi, Kuukausi, Päivä ja Aika -sarakkeet.
    - Poistaa tarpeettomat alkuperäiset aika- ja paikkatiedot.
    - Muuntaa mittausdata numeeriseen muotoon, käsitellen mahdolliset virheet.
    - Pudottaa sarakkeet, jotka ovat kokonaan tyhjiä (vain NaN).
    - Nimetä sarakkeet selkeiksi, englanninkielisiksi nimiksi analyysin helpottamiseksi
      ja lisätä päätteitä (_W1, _W2) erottamaan samankaltaiset sarakkeet eri
      lähdetiedostoista.
 3. Yhdistää esikäsitellyt sää- ja ilmanlaatudatat yhdeksi kattavaksi Pandas
    DataFrameksi aikaleiman perusteella.
 4. Tunnistaa ja käsitellä mahdolliset duplikaattiaikaleimat, jotka voivat syntyä
    esimerkiksi datan keruussa tai yhdistämisessä. Duplikaatit käsitellään
    laskemalla päällekkäisten rivien numeeristen arvojen keskiarvo.
 5. Varmistaa, että lopullisessa DataFramessa on tasainen tunnin välein etenevä
    aikasarjaindeksi (`freq='h'`) koko data-ajan mitalta. Tämä täyttää mahdolliset
    puuttuvat tunnit NaN-arvoilla.
 6. Täyttää kaikki jäljelle jääneet puuttuvat arvot (NaN) käyttäen ensin
    eteenpäin täyttöä (`ffill`) ja sitten taaksepäin täyttöä (`bfill`).
 7. (Valinnaisesti) Tallentaa lopullisen, siivotun ja täydennetyn DataFramen
    tehokkaaseen Parquet-tiedostomuotoon, joka soveltuu hyvin jatkokäyttöön.

 Vaaditut kirjastot:
 -------------------
 - pandas: Datan käsittely ja analysointi (DataFrame).
 - numpy: Numeerinen laskenta (käytetään esim. NaN-arvojen esitykseen).
 - requests: HTTP-pyyntöjen tekeminen (datan lataus URL:sta).
 - io: Tekstipohjaisen datan käsittely muistissa (CSV-datan luku).
 - os: Käyttöjärjestelmäriippumattomat polkujen käsittelytoiminnot (tallennus).
 - pyarrow: Vaaditaan Parquet-tiedostojen kirjoittamiseen (`pip install pyarrow`).

 Käyttö:
 -------
 1. Varmista, että vaaditut kirjastot on asennettu.
 2. Aja skripti Python-ympäristössä.
 3. Tarkkaile tulosteita varmistaaksesi, että kaikki vaiheet suoritetaan odotetusti.
    Erityisesti sarakkeiden uudelleennimeämiskohdat Vaiheessa 2 saattavat vaatia
    säätöä, jos lähde-CSV:n sarakkeiden nimet muuttuvat.
 4. Jos haluat tallentaa lopputuloksen, poista kommenttimerkit Vaiheen 5
    tallennuskoodista ja varmista, että 'pyarrow' on asennettu.
"""

# === Vaihe 1: Alustus ja Kirjastojen Tuonti ===
print("="*60)
print(" Vaihe 1: Alustus ja Kirjastojen Tuonti")
print("="*60)

# Tuodaan kirjastot selittävin kommentein
import pandas as pd  # Peruskirjasto datan manipulointiin DataFramien avulla
import numpy as np   # Tieteellisen laskennan peruskirjasto, käytetään NaN-arvoihin ym.
import requests      # Mahdollistaa HTTP-pyyntöjen tekemisen, esim. datan lataus verkosta
import io            # Mahdollistaa merkkijonojen käsittelyn tiedostojen kaltaisina virtauksina
import os            # Tarjoaa käyttöjärjestelmäriippumattomia toimintoja, kuten polkujen käsittely

# Määritellään datatiedostojen URL-osoitteet GitHubin raakasisältöön.
# Nämä ovat suoria linkkejä itse tiedostoon, eivät GitHubin web-sivulle.
url_weather_1 = "https://raw.githubusercontent.com/rrwiren/ilmanlaatu-ennuste-helsinki/main/data/raw/Helsinki%20Kaisaniemi_%2011.4.2023%20-%2011.4.2025_4c0a0316-74e0-4792-9854-fd6315fbc965.csv"
url_weather_2 = "https://raw.githubusercontent.com/rrwiren/ilmanlaatu-ennuste-helsinki/main/data/raw/Helsinki%20Kaisaniemi_%2011.4.2023%20-%2011.4.2025_84370efb-e7a0-48ed-b991-0432c84c1475.csv"
url_aq = "https://raw.githubusercontent.com/rrwiren/ilmanlaatu-ennuste-helsinki/main/data/raw/Helsinki%20Kallio%202_%2011.4.2023%20-%2011.4.2025_8bbe3500-d31d-459d-ac37-16b4bd64a2cc.csv"

# === Apufunktio CSV-datan lataamiseen ja alustavaan lukuun ===
def load_csv_from_url(url):
    """
    Lataa CSV-tiedoston annetusta URL-osoitteesta ja yrittää lukea sen DataFrameksi.

    Args:
        url (str): Ladattavan CSV-tiedoston URL-osoite.

    Returns:
        pandas.DataFrame or None: Ladattu data DataFrame-objektina, tai None jos epäonnistui.
    """
    print(f"\nYritetään ladata ja lukea: {url}")
    try:
        response = requests.get(url)
        response.raise_for_status() # Tarkistaa HTTP-statuskoodin (esim. 200 OK)
        csv_data = io.StringIO(response.text) # Luetaan tekstisisältö muistiin virtana
        # Yritetään lukea pilkulla erotettuna (perustuen aiempaan onnistumiseen)
        df = pd.read_csv(csv_data, sep=',')
        print(f"  -> Luettu onnistuneesti (oletus sep=',')")
        return df
    except requests.exceptions.RequestException as e:
        print(f"  -> VIRHE LADATESSA: {e}")
        return None
    except Exception as e: # Muu virhe (esim. CSV-luvussa)
        print(f"  -> VIRHE LUETTAESSA CSV: {e}")
        # Tässä voisi olla lisälogiikkaa kokeilla eri erottimia (';') tai enkoodauksia ('latin1')
        return None

# === Ladataan kaikki kolme dataa ===
print("\nLadataan datatiedostot...")
df_w1_raw = load_csv_from_url(url_weather_1)
df_w2_raw = load_csv_from_url(url_weather_2)
df_aq_raw = load_csv_from_url(url_aq)

# Tarkistetaan latausten onnistuminen
if df_w1_raw is None or df_w2_raw is None or df_aq_raw is None:
    print("\nKRIITTINEN VIRHE: Kaikkia tiedostoja ei voitu ladata. Tarkista URL-osoitteet ja verkkoyhteys. Keskeytetään.")
    # exit() # Voit aktivoida tämän, jos haluat skriptin pysähtyvän tähän.
else:
    print("\nKaikki tiedostot ladattu onnistuneesti. Jatketaan esikäsittelyyn.")
    data_loaded_successfully = True # Asetetaan lippu jatkoa varten

# Suoritetaan loput vain, jos lataus onnistui
if data_loaded_successfully:

    #==============================================================================
    print("\n" + "="*60)
    print(" Vaihe 2: Datan Esikäsittely (jokainen tiedosto erikseen)")
    print("="*60)
    #==============================================================================

    # --- Apufunktio FMI-datan yleiseen esikäsittelyyn ---
    def preprocess_fmi_data(df, data_type_name):
        """
        Suorittaa FMI:n CSV-datalle yleiset esikäsittelyvaiheet.

        Args:
            df (pandas.DataFrame): DataFrame, joka sisältää FMI:n raakadataa.
            data_type_name (str): Datan kuvaava nimi tulosteita varten.

        Returns:
            pandas.DataFrame or None: Esikäsitelty DataFrame tai None, jos virhe.
        """
        if df is None:
            return None

        print(f"\n--- Esikäsitellään: {data_type_name} ---")
        original_cols = df.columns.tolist()
        print("Alkuperäiset sarakkeet:", original_cols)

        # --- 1. Aikaleiman muodostus ---
        # Oletetaan FMI:n käyttävän näitä sarakkeita päivämäärän ja ajan esittämiseen.
        date_cols = ['Vuosi', 'Kuukausi', 'Päivä']
        time_col = 'Aika [Paikallinen aika]'
        # Varmistetaan, että kaikki tarvittavat sarakkeet ovat olemassa.
        if all(col in original_cols for col in date_cols) and time_col in original_cols:
            try:
                # Yhdistetään sarakkeet muotoon "YYYY-MM-DD HH:MM:SS" merkkijonoksi.
                # zfill(2) lisää etunollan tarvittaessa (esim. 1 -> 01).
                datetime_str_series = df['Vuosi'].astype(str) + '-' + \
                                      df['Kuukausi'].astype(str).str.zfill(2) + '-' + \
                                      df['Päivä'].astype(str).str.zfill(2) + ' ' + \
                                      df[time_col].astype(str)
                # Muunnetaan merkkijonot datetime-objekteiksi. Pandas tunnistaa muodon automaattisesti.
                df['Timestamp'] = pd.to_datetime(datetime_str_series)
                print("  -> Aikaleima ('Timestamp') muodostettu onnistuneesti.")
                # Pudotetaan alkuperäiset aika/pvm-sarakkeet ja vakio 'Havaintoasema'-sarake.
                cols_to_drop = date_cols + [time_col]
                if 'Havaintoasema' in original_cols:
                     cols_to_drop.append('Havaintoasema')
                df = df.drop(columns=cols_to_drop, errors='ignore')
            except Exception as e:
                print(f"  -> VIRHE aikaleiman muodostamisessa: {e}. Tarkista datan muoto.")
                return None # Palautetaan None, koska ilman aikaleimaa ei voida jatkaa.
        else:
            missing_cols = [col for col in date_cols + [time_col] if col not in original_cols]
            print(f"  -> VIRHE: Tarvittavia aikaleimasarakkeita ei löytynyt. Puuttuvat: {missing_cols}")
            return None

        # --- 2. Indeksin asetus ---
        # Asetetaan juuri luotu 'Timestamp' DataFramen indeksiksi. Tämä on standardikäytäntö
        # aikasarjadatan käsittelyssä Pandasilla, ja se helpottaa yhdistämistä ja aikaperusteisia operaatioita.
        try:
            df = df.set_index('Timestamp')
            print("  -> 'Timestamp' asetettu indeksiksi.")
        except KeyError:
            print("  -> VIRHE: 'Timestamp'-saraketta ei löytynyt indeksiksi asettamista varten.")
            return None

        # --- 3. Numeeriseksi muunto ---
        # Muunnetaan kaikki jäljellä olevat sarakkeet numeerisiksi (float tai int).
        # Tämä on tärkeää laskentaa ja mallinnusta varten.
        print("  -> Muunnetaan data numeeriseksi (virheet -> NaN)...")
        for col in df.columns:
            if not pd.api.types.is_numeric_dtype(df[col]):
                # errors='coerce' korvaa arvot, joita ei voi muuntaa, NaN-arvolla.
                df[col] = pd.to_numeric(df[col], errors='coerce')
        print("    -> Muunnos valmis.")

        # --- 4. Täysin tyhjien sarakkeiden poisto ---
        # Joskus FMI-data voi sisältää sarakkeita, joissa ei ole lainkaan mittauksia (vain NaN).
        # Nämä poistetaan turhina. axis=1 tarkoittaa sarakkeita, how='all' tarkoittaa "poista vain jos kaikki arvot ovat NaN".
        df = df.dropna(axis=1, how='all')
        print(f"  -> Lopulliset sarakkeet esikäsittelyn jälkeen: {df.columns.tolist()}")

        return df

    # --- Sovelletaan esikäsittelyä ja uudelleennimeämistä ---

    # Säädata 1 (Kaisaniemi)
    df_w1 = preprocess_fmi_data(df_w1_raw, "Säädata 1 (Kaisaniemi)")
    if df_w1 is not None:
        # Määritellään halutut uudet nimet (englanniksi) alkuperäisille suomenkielisille nimille.
        # Käytä _W1-päätettä erottamaan mahdolliset päällekkäiset sarakkeet toisesta säädatatiedostosta.
        # TÄRKEÄÄ: Tarkista, että avaimet (vasen puoli) vastaavat TARKALLEEN datassa olevia nimiä!
        rename_map_w1 = {
            'Ilman lämpötila keskiarvo [°C]': 'Temperature_W1',
            'Tuulen suunta keskiarvo [°]': 'WindDirection', # Oletetaan tämä uniikiksi tiedostolle 1
            'Keskituulen nopeus keskiarvo [m/s]': 'WindSpeed_W1',
            'Näkyvyys keskiarvo [m]': 'Visibility',
            'Pilvisyys [1/8]': 'Cloudiness', # Huom: Tämä saattaa puuttua, jos se oli tyhjä
            'Ilmanpaine merenpinnan tasolla keskiarvo [hPa]': 'Pressure_SeaLevel'
        }
        # Suoritetaan uudelleennimeäminen vain niille sarakkeille, jotka löytyvät datasta.
        valid_rename_map_w1 = {k: v for k, v in rename_map_w1.items() if k in df_w1.columns}
        df_w1 = df_w1.rename(columns=valid_rename_map_w1)
        print("  -> Säädata 1 uudelleennimetty. Sarakkeet:", df_w1.columns.tolist())
        print("     Esimerkkirivi:\n", df_w1.head(1).to_string()) # Tulostetaan 1 rivi luettavammin

    # Säädata 2 (Kaisaniemi)
    df_w2 = preprocess_fmi_data(df_w2_raw, "Säädata 2 (Kaisaniemi)")
    if df_w2 is not None:
        # Vastaava uudelleennimeäminen toiselle säädatatiedostolle (_W2-pääte).
        rename_map_w2 = {
            'Lämpötilan keskiarvo [°C]': 'Temperature_W2',
            'Ylin lämpötila [°C]': 'Temperature_Max',
            'Alin lämpötila [°C]': 'Temperature_Min',
            'Keskituulen nopeus [m/s]': 'WindSpeed_W2',
            'Tuulen suunnan keskiarvo [°]': 'WindDirection_W2',
            'Ilmanpaineen keskiarvo [hPa]': 'Pressure_W2'
        }
        valid_rename_map_w2 = {k: v for k, v in rename_map_w2.items() if k in df_w2.columns}
        df_w2 = df_w2.rename(columns=valid_rename_map_w2)
        print("  -> Säädata 2 uudelleennimetty. Sarakkeet:", df_w2.columns.tolist())
        print("     Esimerkkirivi:\n", df_w2.head(1).to_string())

    # Ilmanlaatu (Kallio 2)
    df_aq = preprocess_fmi_data(df_aq_raw, "Ilmanlaatu (Kallio 2)")
    if df_aq is not None:
        # Määritellään, mitkä ilmanlaatusarakkeet halutaan mukaan ja millä nimillä.
        aq_cols_to_keep_and_rename = {
            'Otsoni [µg/m3]': 'Ozone', # TÄRKEIN: Kohdemuuttuja
            'Hengitettävät hiukkaset <10 µm [µg/m3]': 'PM10',
            'Pienhiukkaset <2.5 µm [µg/m3]': 'PM25',
            'Typpidioksidi [µg/m3]': 'NO2',
            'Typpimonoksidi [µg/m3]': 'NO',
            'Hiilimonoksidi [µg/m3]': 'CO',
            'Rikkidioksidi [µg/m3]': 'SO2',
            'Musta hiili [µg/m3]': 'BlackCarbon'
        }
        # Valitaan vain ne sarakkeet, jotka löytyvät datasta
        cols_to_select = [k for k in aq_cols_to_keep_and_rename.keys() if k in df_aq.columns]
        # Erityinen tarkistus kohdemuuttujalle (Ozone)
        if 'Otsoni [µg/m3]' not in cols_to_select:
             print("  -> KRIITTINEN VIRHE: Otsonisaraketta 'Otsoni [µg/m3]' ei löytynyt ilmanlaatudatasta!")
             df_aq = None # Estetään jatko
        else:
            df_aq_filtered = df_aq[cols_to_select] # Valitaan halutut sarakkeet
            valid_rename_map_aq = {k: v for k, v in aq_cols_to_keep_and_rename.items() if k in cols_to_select}
            df_aq = df_aq_filtered.rename(columns=valid_rename_map_aq) # Nimetään ne uudelleen
            print("  -> Ilmanlaatudata suodatettu ja uudelleennimetty. Sarakkeet:", df_aq.columns.tolist())
            print("     Esimerkkirivi:\n", df_aq.head(1).to_string())

    #==============================================================================
    print("\n" + "="*60)
    print(" Vaihe 3: DataFramesien Yhdistäminen")
    print("="*60)
    #==============================================================================

    # Tarkistetaan taas, että kaikki tarvittavat DataFramet ovat valmiina yhdistämistä varten
    if df_w1 is not None and df_w2 is not None and df_aq is not None:

        # --- Yhdistetään ensin kaksi säädataa ---
        print("\nYhdistetään säädatat (df_w1 ja df_w2) indeksin perusteella...")
        # how='outer' säilyttää kaikki rivit molemmista. Jos aikaleima on vain toisessa,
        # toisen sarakkeisiin tulee NaN. Jos sama aikaleima on molemmissa, ne yhdistyvät samalle riville.
        df_weather_combined = pd.merge(df_w1, df_w2, left_index=True, right_index=True, how='outer', suffixes=('_Check1', '_Check2'))
        print(f"  -> Yhdistetyn säädatan muoto: {df_weather_combined.shape}")
        print(f"  -> Sarakkeet: {df_weather_combined.columns.tolist()}")
        # Tarkistetaan, syntyikö yhdistämisessä päällekkäisiä sarakkeita (ei pitäisi, koska käytimme päätteitä)
        if any('_Check' in col for col in df_weather_combined.columns):
            print("  -> Varoitus: Yhdistämisessä syntyi päällekkäisiä sarakkeita? Tarkista sarakelistat.")

        # --- Yhdistetään yhdistetty säädata ilmanlaatutataan ---
        print("\nYhdistetään sää- ja ilmanlaatutata (df_aq) indeksin perusteella...")
        # Käytetään jälleen 'outer' mergeä, jotta kaikki uniikit ajankohdat säilyvät.
        df_final = pd.merge(df_weather_combined, df_aq, left_index=True, right_index=True, how='outer', suffixes=('_Weather', '_AQ'))
        print(f"  -> Lopullisen yhdistetyn datan muoto: {df_final.shape}")
        print(f"  -> Lopulliset sarakkeet: {df_final.columns.tolist()}")

        #==========================================================================
        print("\n" + "="*60)
        print(" Vaihe 4: Lopullinen Siivous ja NaN-arvojen Käsittely")
        print("="*60)
        #==========================================================================

        # --- 1. Aikajärjestys ---
        # Varmistetaan, että data on aikajärjestyksessä, mikä on tärkeää aikasarjaoperaatioille.
        df_final = df_final.sort_index()
        print("\nDataFrame lajiteltu aikaleiman ('Timestamp' indeksi) mukaan.")

        # --- 2. Duplikaatti-indeksien käsittely ---
        # Tarkistetaan, onko sama aikaleima useamman kerran indeksissä.
        if not df_final.index.is_unique:
            num_duplicates = df_final.index.duplicated().sum()
            print(f"\nVAROITUS: Löydettiin {num_duplicates} duplikaatti-indeksiä!")
            print("  -> Käsitellään duplikaatit laskemalla keskiarvo...")
            # Ryhmitellään indeksin mukaan ja lasketaan keskiarvo kustakin sarakkeesta
            # Tämä yhdistää duplikaattirivit yhdeksi riviksi keskiarvoilla.
            df_final = df_final.groupby(level=0).mean()
            print(f"  -> Duplikaatit käsitelty. DataFramen uusi muoto: {df_final.shape}")
            if not df_final.index.is_unique:
                 print("  -> KRIITTINEN VIRHE: Duplikaatteja jäi vielä käsittelyn jälkeen!")
            else:
                 print("  -> Duplikaatti-indeksit poistettu onnistuneesti.")
        else:
            print("\nIndeksi on uniikki, ei duplikaatteja.")

        # --- 3. Tasaisen tuntifrekvenssin varmistaminen ---
        print("\nVarmistetaan tasainen tuntifrekvenssi ('freq=h')...")
        if not df_final.empty:
          min_ts, max_ts = df_final.index.min(), df_final.index.max()
          print(f"  -> Datan aika-alue: {min_ts} - {max_ts}")
          # Luodaan täydellinen tuntiperusteinen aikaindeksi datan alku- ja loppuhetken välille.
          full_time_range = pd.date_range(start=min_ts, end=max_ts, freq='h')
          print(f"  -> Odotettu tuntien määrä tällä aikavälillä: {len(full_time_range)}")
          # Uudelleenindeksoidaan DataFrame tällä täydellisellä indeksillä.
          # Mahdolliset puuttuvat tunnit lisätään NaN-arvoilla.
          df_final = df_final.reindex(full_time_range)
          print(f"  -> DataFrame uudelleenindeksoitu. Uusi muoto: {df_final.shape}")
        else:
          print("  -> DataFrame on tyhjä, ei voida uudelleenindeksoida.")

        # --- 4. Puuttuvien arvojen (NaN) käsittely ---
        print("\nKäsitellään puuttuvat arvot (NaN)...")
        if not df_final.empty:
          nan_counts_before = df_final.isnull().sum()
          print("  -> NaN-arvojen määrä / sarake ENNEN täyttöä:")
          print(nan_counts_before[nan_counts_before > 0].to_string())

          # Käytetään ffill() (täyttö edellisellä arvolla) ja sen jälkeen bfill() (täyttö seuraavalla).
          # ffill on yleinen aikasarjoille, koska se ei käytä tulevaisuuden tietoa.
          # bfill lisätään täyttämään mahdolliset NaN:t datan ihan alussa.
          # Huom: Tämä on yksinkertainen strategia. Monimutkaisempia (esim. interpolointi)
          # voitaisiin harkita, mutta tämä on hyvä lähtökohta.
          df_final = df_final.ffill().bfill()

          nan_counts_after = df_final.isnull().sum().sum()
          if nan_counts_after == 0:
              print("\n  -> Kaikki NaN-arvot täytetty onnistuneesti (ffill + bfill).")
          else:
              print(f"\n  -> VAROITUS: {nan_counts_after} NaN-arvoa jäi täytön jälkeen! Tarkista data.")
              print("     Sarakkeet joissa NaN:\n", df_final.isnull().sum()[df_final.isnull().sum() > 0])
        else:
          print("  -> DataFrame on tyhjä, NaN-arvojen käsittelyä ei suoriteta.")

        # --- 5. Lopullinen tarkistus ja yhteenveto ---
        if not df_final.empty:
          print("\n" + "-"*40)
          print(" Lopullisen DataFramen Yhteenveto")
          print("-"*40)
          print("\nEnsimmäiset 5 riviä:")
          print(df_final.head())
          print("\nViimeiset 5 riviä:")
          print(df_final.tail())
          print("\nPerustilastot (describe):")
          print(df_final.describe())
          print("\nInfo (sarakkeet, datatyypit, muisti):")
          df_final.info()
        else:
            print("\nLopullinen DataFrame on tyhjä.")

        #==========================================================================
        print("\n" + "="*60)
        print(" Vaihe 5: Tallenna Käsitelty Data Parquet-muodossa (Valinnainen)")
        print("="*60)
        #==========================================================================
        # Tallennetaan lopputulos Parquet-tiedostoon. Parquet on tehokas, sarakepohjainen
        # tallennusmuoto, joka säilyttää datatyypit hyvin ja on usein nopeampi lukea
        # kuin CSV, erityisesti suurilla data-aineistoilla.
        if not df_final.empty:
          # Tarkistetaan, onko 'pyarrow'-kirjasto asennettu, sitä tarvitaan Parquet-kirjoitukseen.
          try:
              import pyarrow
              print("\n'pyarrow'-kirjasto löytyy. Parquet-tallennus jatkuu.")
              pyarrow_installed = True
          except ImportError:
              print("\nVAROITUS: 'pyarrow'-kirjastoa ei löydy. Parquet-tallennus vaatii sen.")
              print("          Voit asentaa sen komentoriviltä komennolla: pip install pyarrow")
              print("          tai notebookissa: !pip install pyarrow")
              pyarrow_installed = False

          if pyarrow_installed:
              # Määritellään tallennushakemisto ja tiedostonimi.
              output_folder = "data/processed"
              output_filename = "Helsinki_Kaisaniemi_Kallio_Combined_Processed_v2.parquet" # Lisätty versio nimeen
              output_path_parquet = os.path.join(output_folder, output_filename)

              try:
                  # Luodaan 'data/processed' -hakemisto, jos sitä ei ole olemassa.
                  # exist_ok=True estää virheen, jos hakemisto on jo olemassa.
                  os.makedirs(output_folder, exist_ok=True)

                  # Tallennetaan DataFrame Parquet-tiedostoon.
                  # index=True varmistaa, että Timestamp-indeksi tallennetaan mukaan.
                  df_final.to_parquet(output_path_parquet, index=True)
                  print(f"\nKäsitelty data tallennettu Parquet-tiedostoon: {output_path_parquet}")
              except Exception as e:
                  # Tulostetaan virhe, jos tallennus epäonnistuu.
                  print(f"\nVIRHE tallennettaessa käsiteltyä dataa Parquet-muodossa: {e}")
          else:
              print("\nParquet-tallennusta ei suoritettu, koska 'pyarrow'-kirjasto puuttuu.")

    else: # Tämä else-haara ajetaan, jos jokin df_w1, df_w2 tai df_aq oli None yhdistämisvaiheessa
        print("\nYhdistämistä tai lopullista siivousta ei voitu suorittaa, koska yksi tai useampi DataFrame puuttui.")

    print("\n" + "="*60)
    print(" Skriptin suoritus päättyi")
    print("="*60)

# Tämä else-haara ajetaan, jos alkuperäinen datan lataus epäonnistui
elif not data_loaded_successfully:
    print("\nSkriptin suoritus keskeytettiin datan latausvirheen vuoksi.")